In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import backtrader as bt
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go

# Initialize Dash app
app = dash.Dash(__name__)

# Default strategy class
class DefaultStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=15)

    def next(self):
        if self.sma > self.data.close:
            if not self.position:
                self.buy()
        elif self.sma < self.data.close:
            if self.position:
                self.sell()

# Function to download historical data
def get_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    data = bt.feeds.PandasData(dataname=data)
    return data

# Function to run backtest and return results
def run_backtest(strategy_class, data):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy_class)
    cerebro.adddata(data)
    cerebro.broker.set_cash(10000)
    cerebro.broker.set_commission(commission=0.001)
    cerebro.addobserver(bt.observers.DrawDown)
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.TimeReturn)
    
    cerebro.run()
    portfolio_value = cerebro.broker.getvalue()
    return portfolio_value, cerebro

# Dash Layout
app.layout = html.Div([
    html.H1("Trading Bot Configuration", style={'textAlign': 'center'}),
    
    dcc.Input(id='symbol', type='text', placeholder='Enter Stock Symbol (e.g., AAPL)', style={'width': '40%'}),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date='2020-01-01',
        end_date='2021-01-01',
        display_format='YYYY-MM-DD'
    ),
    html.Button('Run Backtest', id='run-btn', n_clicks=0),
    
    html.Div(id='backtest-output'),
    dcc.Graph(id='strategy-plot')
])

# Dash Callback
@app.callback(
    [Output('backtest-output', 'children'),
     Output('strategy-plot', 'figure')],
    [Input('run-btn', 'n_clicks')],
    [Input('symbol', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_backtest_output(n_clicks, symbol, start_date, end_date):
    if n_clicks > 0 and symbol:
        # Get data and run backtest
        data = get_data(symbol, start_date, end_date)
        portfolio_value, cerebro = run_backtest(DefaultStrategy, data)
        
        # Backtest output text
        output_text = f"Portfolio Value: ${portfolio_value:,.2f}"
        
        # Prepare strategy plot
        plt_data = pd.DataFrame(cerebro.broker.get_value(), columns=['Portfolio Value'])
        fig = go.Figure(data=[go.Scatter(x=plt_data.index, y=plt_data['Portfolio Value'], mode='lines', name='Portfolio Value')])
        fig.update_layout(title=f"Backtest Result for {symbol}", xaxis_title="Date", yaxis_title="Portfolio Value")
        
        return output_text, fig
    return "Please enter a valid symbol and date range.", {}

# Run app in notebook
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8050)

### Imports
import dash:
Imports the Dash framework, which is used to create interactive web applications in Python.

from dash import dcc, html:
Imports dcc (Dash Core Components) and html (HTML components). dcc is used for interactive elements like graphs and inputs, while html is used for static HTML elements like headings and divs.

from dash.dependencies import Input, Output:
Imports Input and Output from dash.dependencies. These are used to define how different components (like buttons and inputs) interact with each other in Dash by specifying what triggers changes (Input) and what components will change (Output).

import backtrader as bt:
Imports the backtrader library as bt. Backtrader is used for backtesting trading strategies.

import yfinance as yf:
Imports yfinance as yf, a library used to download historical financial data from Yahoo Finance.

import pandas as pd:
Imports the pandas library as pd. Pandas is used for data manipulation and analysis, especially for working with time series data.
import plotly.graph_objs as go:

Imports plotly.graph_objs as go. Plotly is used for creating interactive visualizations. graph_objs contains the building blocks to create charts like scatter plots.

app = dash.Dash(__name__):
Initializes a new Dash app. The __name__ argument tells Dash to run the app when the script is executed. This app object is the main entry point for creating the web application.

### Code Description
The provided code sets up an interactive web application using the Dash framework, which is built on top of Flask and Plotly. The app allows users to input stock symbols, select a date range, and run a backtest using a predefined trading strategy.

First, the code imports essential libraries. Dash, dash_core_components (dcc), and dash_html_components (html) are used to build the app's user interface. The dash.dependencies module provides the Input and Output components to define the interactivity between UI elements, while backtrader is used for the trading strategy, and yfinance retrieves historical stock data. pandas helps manipulate the stock data, and plotly.graph_objs is used to create interactive plots of the backtest results.

The Dash app is initialized with app = dash.Dash(__name__), creating a new Dash application. A DefaultStrategy class is defined, which inherits from backtrader.Strategy. This strategy uses a 15-day Simple Moving Average (SMA) to determine whether to buy or sell a stock. The next() method is called on each step of the backtest, making buy or sell decisions based on whether the current stock price is above or below the SMA.

The get_data function uses yfinance to download historical stock data between the specified start and end dates. This data is then converted into a format that is compatible with backtrader using bt.feeds.PandasData(dataname=data). The run_backtest function sets up the backtest by initializing the Cerebro engine from backtrader, adding the strategy, data, and various backtest parameters like initial cash and commission rates. After running the backtest, it returns the final portfolio value and the Cerebro engine, which contains the backtest results.

The layout of the app is created using Dash components. It includes an input field (dcc.Input) for the stock symbol, a date range picker (dcc.DatePickerRange) for selecting the backtest period, and a button (html.Button) to trigger the backtest. The output of the backtest, including the portfolio value and the plot, is displayed using html.Div and dcc.Graph.

The @app.callback decorator defines the interaction between the input components (stock symbol, date range, and button click) and the output components (portfolio value and plot). The callback function is triggered when the button is clicked, it fetches the stock data, runs the backtest using the DefaultStrategy, and returns the portfolio value along with the plot. The portfolio value is displayed in a Div and the plot is shown in the Graph component using Plotly.

Finally, the app.run_server(debug=True, use_reloader=False, port=8050) command starts the Dash app on port 8050. The app runs in debug mode, which helps during development, and the use_reloader=False argument prevents the app from automatically reloading itself.